In [45]:
import numpy as np
import os
import matplotlib.image as mpimg

In [46]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.losses import binary_crossentropy

In [47]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [48]:
def pick_test_images():
    test_imgs = []
    for i in range(1, 51):
        name = '../Data/test_set_images/test_'+str(i)+'/test_' + str(i) + '.png'
        test_imgs.append(load_image(name))
    return test_imgs
        
test = pick_test_images()

In [56]:
def train(X, Y):
        reg = 1e-6
        pool_size = (2, 2)
        nb_classes = 2
        patch_size = 16

        if K.image_dim_ordering() == 'th':
            input_shape = (3, patch_size, patch_size)
        else:
            input_shape = (patch_size, patch_size, 3)
    
        model = Sequential()

        model.add(Convolution2D(32, kernel_size=(3, 3), 
                                input_shape = ( patch_size, patch_size, 3), 
                                activation = 'relu',
                                padding = 'SAME'))
        model.add(Flatten())
        model.add(Dense(units = 128, activation = 'relu'))
        model.add(Dense(units = 2, activation = 'sigmoid'))
        
        print('Training set shape: ', X.shape) 
            
        batch_size = 200
        nb_classes = 2
        nb_epoch = 10
        steps_per_epoch = 10


        opt = Adam(lr=0.001) # Adam optimizer with default initial learning rate

        np.random.seed(3) # Ensure determinism
        
        def generate_minibatch():
            """
            Procedure for real-time minibatch creation and image augmentation.
            This runs in a parallel thread while the model is being trained.
            """
            while 1:
                # Generate one minibatch
                X_batch = np.empty((batch_size, patch_size, patch_size, 3))
                Y_batch = np.empty((batch_size, 2))
                for i in range(batch_size):
                    # Select a random image
                    idx = np.random.choice(X.shape[0])
                    X_batch[i] = X[idx]
                    Y_batch[i] = Y[idx]
                
                if K.image_dim_ordering() == 'th':
                    X_batch = np.rollaxis(X_batch, 3, 1)
                    
                yield (X_batch, Y_batch)

        # This callback reduces the learning rate when the training accuracy does not improve any more
        lr_callback = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=5,
                                        verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
        
        # Stops the training process upon convergence
        stop_callback = EarlyStopping(monitor='acc', min_delta=0.0001, patience=11, verbose=1, mode='auto')
        
        model.compile(loss=binary_crossentropy,
                      optimizer=opt,
                      metrics=['accuracy'])

        try:
            model.fit_generator(generate_minibatch(),
                            steps_per_epoch=steps_per_epoch,
                            nb_epoch=nb_epoch,
                            verbose=1,
                            callbacks=[lr_callback, stop_callback])
        except KeyboardInterrupt:
            # Do not throw away the model in case the user stops the training process
            pass

        print('Training completed')
        
        model.save_weights('pesi')
        
        return model

In [57]:
# Loaded a set of image

root_dir = "../Data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

print(np.array(imgs).shape)

Loading 100 images
satImage_001.png
Loading 100 images
satImage_001.png
(100, 400, 400, 3)


In [58]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
X = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0
    
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

# Print feature statistics

print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))  #TODO: fix, length(unique(Y)) 

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

Computed 62500 features
Feature dimension = 16
Number of classes = 1
Class 0: 46309 samples
Class 1: 16191 samples


In [59]:
model = train(X,Y)

Training set shape:  (62500, 16, 16, 3)


C:\Users\nicol\Anaconda3\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=10, verbose=1, callbacks=[<keras.ca..., epochs=10)`


Epoch 1/10
10/10 [==============================] - 2s 243ms/step - loss: 0.5943 - acc: 0.7183
Epoch 2/10
10/10 [==============================] - 1s 105ms/step - loss: 0.5582 - acc: 0.7530
Epoch 3/10
10/10 [==============================] - 1s 111ms/step - loss: 0.5312 - acc: 0.7645
Epoch 4/10
10/10 [==============================] - 1s 110ms/step - loss: 0.5446 - acc: 0.7485
Epoch 5/10
10/10 [==============================] - 1s 112ms/step - loss: 0.5328 - acc: 0.7450
Epoch 6/10
10/10 [==============================] - 1s 109ms/step - loss: 0.5320 - acc: 0.7488
Epoch 7/10
10/10 [==============================] - 1s 110ms/step - loss: 0.5372 - acc: 0.7470
Epoch 8/10
10/10 [==============================] - 1s 108ms/step - loss: 0.5403 - acc: 0.7410

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 9/10
10/10 [==============================] - 1s 117ms/step - loss: 0.5344 - acc: 0.7455
Epoch 10/10
10/10 [==============================] - 1s 120ms/st

In [68]:
test_images = pick_test_images()

print(np.array(test_images).shape)

n_test = len(test_images)

test_patches = [img_crop(test_images[i], patch_size, patch_size) for i in range(n_test)]

test_patches = np.asarray([test_patches[i][j] for i in range(len(test_patches)) for j in range(len(test_patches[i]))])

(50, 608, 608, 3)


In [70]:
Z = model.predict(test_patches)
labels = (Z[:,0] < Z[:,1]) * 1
labels.shape

(72200,)

In [71]:
def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

def MY_mask_to_submission_strings(mask1D, fileNumber):
    img_number = fileNumber
    patch_size = 16
    test_size = 608
    im = label_to_img(test_size, test_size, patch_size, patch_size, mask1D)
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number+1, j, i, label))


def MY_masks_to_submission(submission_filename, masks1D):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for idx,fn in enumerate(masks1D):
            f.writelines('{}\n'.format(s) for s in MY_mask_to_submission_strings(fn,idx))
            
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0

In [72]:
labels = labels.reshape(50,-1)
print(labels.shape)


(50, 1444)


In [74]:
MY_masks_to_submission('dummy_submission.csv', labels)